In [ ]:
from itertools import islice, cycle
from tau_profile_parser import TauProfileParser
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import os
import re
import glob

### Parsing MULTI__ files

The TauProfileParser has been updated to parse MULTI files. However, by default it will look for profile.* files before looking for MULTI folders. By passing the `True` flag for MULTI it will automatically parse the MULTI folders.

In [ ]:
path_to_files = 'local/path/to/tau2/examples/jupyter_notebooks/ExampleData/theta_64_flash/nocall'
multi_files = TauProfileParser.parse(path_to_files,MULTI=True)

In [ ]:
multi_files.metadata

### Multi-level Column Names
When MULTI files are parsed, the column names are organized with a hierarchy. The top level (level 0) are the Intervals. The second level is the the subdivision of exclusive and inclusive into the metrics. Since all metrics have the much of the same data, only the Exclusive and Inclusive are differentiated. The index of the dataframe is also multi index: `Node,Context,Thread,Timer` being the order.

In [ ]:
multi_files.interval_data().head()

In [ ]:
exclusive_df = multi_files.interval_data()['Exclusive']
exclusive_df.head()

In [ ]:
max_dict = exclusive_df.max().to_dict()

In [ ]:
normalized_df = exclusive_df.copy()
for columns in exclusive_df.columns:
    normalized_df[columns] = normalized_df[columns].apply(lambda x: x/max_dict[columns])


In [ ]:
flat_index = normalized_df.index.to_flat_index()
threads = [tup[2] for tup in flat_index]
timer = [tup[3] for tup in flat_index]

normalized_df.reset_index(inplace=True)
normalized_df.sort_values(('TIME'), ascending=False)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=normalized_df['TIME'].sort_values(ascending=False), 
    y=normalized_df['PAPI_TOT_INS'].sort_values(ascending=False), 
    mode='markers'))

fig.update_layout(updatemenus=
                 [dict(
                 buttons=list([
                     dict(
                     args=['y', [normalized_df.PAPI_L1_DCM]],
                     label = 'PAPI_L1_DCM',
                     method='restyle'),
                     dict(
                     args=['y', [normalized_df.PAPI_RES_STL]],
                         label='PAPI_RES_STL',
                         method='restyle'),
                 dict(
                     args=['y', [normalized_df.TIME]],
                         label='TIME',
                         method='restyle'),
                 dict(
                     args=['y', [normalized_df.PAPI_TOT_INS]],
                         label='PAPI_TOT_INS',
                         method='restyle')]),
                     direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=.3,
            xanchor="left",
            y=1.1,
            yanchor="top",),
                  
                 dict(
                 buttons=list([
                     dict(
                     args=['x', [normalized_df.PAPI_L1_DCM]],
                     label = 'PAPI_L1_DCM',
                     method='restyle'),
                     dict(
                     args=['x', [normalized_df.PAPI_RES_STL]],
                         label='PAPI_RES_STL',
                         method='restyle'),
                 dict(
                     args=['x', [normalized_df.TIME]],
                         label='TIME',
                         method='restyle'),
                 dict(
                     args=['x', [normalized_df.PAPI_TOT_INS]],
                         label='PAPI_TOT_INS',
                         method='restyle')]),
                     direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1.3,
            xanchor="left",
            y=1.1,
            yanchor="top",),])



fig.update_layout(height=600, width=600,
    annotations=[
        dict(text="Y-axis data:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left"),
        dict(text="x-axis data:", showarrow=False,
        x=.6, y=1.085, yref="paper", align="left")
    ]
)

fig.show()

In [ ]:
fig = px.scatter(normalized_df, x=normalized_df.PAPI_TOT_INS, y=normalized_df.PAPI_RES_STL, size=normalized_df.Thread,
                 hover_data=[normalized_df.Timer], trendline='ols')
fig.show()